In [1]:
import os
from PIL import Image
import numpy as np
import os


X = []
y = []

base_path='D:/Git Uploads/Federated-Learning/test/C1'
source_path=base_path
for child in os.listdir(source_path):
    sub_path = os.path.join(source_path, child)
    if os.path.isdir(sub_path):
        for data_file in os.listdir(sub_path):
            X_i = Image.open(os.path.join(sub_path, data_file))
            X_i = np.array(X_i.resize((120,120))) / 255.0
            X.append(X_i)
            y.append(child)
print(X)
print(y)

[array([[0.01176471, 0.01176471, 0.01176471, ..., 0.        , 0.        ,
        0.        ],
       [0.01176471, 0.01176471, 0.01176471, ..., 0.        , 0.        ,
        0.        ],
       [0.01176471, 0.01176471, 0.01176471, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]]), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0.,

In [2]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)
print(y)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [ ]:
# X_train.shape

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y),
                                                    test_size=0.2, random_state=42)
X_train=X_train.reshape([-1,120,120,1])
X_test=X_test.reshape([-1,120,120,1])


import tensorflow as tf
from tensorflow import keras 
from keras import layers
from keras import models

import random

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

Locmodel = models.Sequential()

Locmodel.add(layers.Conv2D(10, (3,3), activation="relu",
                           input_shape=(120,120,1)))

Locmodel.add(layers.MaxPooling2D((2,2)))


Locmodel.add(layers.Conv2D(64, (3,3), activation="relu"))

Locmodel.add(layers.MaxPooling2D((2,2)))

Locmodel.add(layers.Conv2D(64, (3,3), activation="relu"))

Locmodel.add(layers.Flatten())

Locmodel.add(layers.Dense(64, activation="relu"))

Locmodel.add(layers.Dense(32, activation="relu"))

Locmodel.add(layers.Dense(1, activation="sigmoid")) # sigmoid= range between 0 or 1   , 3 nodes =3 class then activation function should be softmax instead of sigmoid.

Locmodel.summary()


Locmodel.compile(optimizer="adam", loss="binary_crossentropy",metrics=["accuracy"])


Locmodel.fit(X_train, y_train, epochs=2, batch_size=32)

testLoss, testAccuracy = Locmodel.evaluate(X_test, y_test)

print(testAccuracy)

Locmodel.save_weights('./my_checkpoint')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 10)      100       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 10)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 64)        5824      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 43264)             0

In [4]:
y_pred = (Locmodel.predict(X_test) > 0.5).astype("int32")

2/2 [==============================] - 0s 10ms/step


In [ ]:
# !pip install flwr

In [5]:
# Define Flower client
import flwr as fl
class CifarClient(fl.client.NumPyClient):
  def get_parameters(self, config):
    return Locmodel.get_weights()

  def fit(self, parameters, config):
    Locmodel.set_weights(parameters)
    Locmodel.fit(X_train, y_train, epochs=1, batch_size=32)
    return Locmodel.get_weights(), len(X_train), {}

  def evaluate(self, parameters, config):
    Locmodel.set_weights(parameters)
    loss, accuracy = Locmodel.evaluate(X_test, y_test)
    return loss, len(X_test), {"accuracy": accuracy}

# Start Flower client
fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=CifarClient())

INFO flwr 2023-08-13 18:58:00,625 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-08-13 18:58:00,629 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2023-08-13 18:58:00,632 | connection.py:39 | ChannelConnectivity.CONNECTING
DEBUG flwr 2023-08-13 18:58:00,633 | connection.py:39 | ChannelConnectivity.READY
DEBUG flwr 2023-08-13 19:02:18,727 | connection.py:113 | gRPC channel closed


KeyboardInterrupt: 

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y),
                                                    test_size=0.2, random_state=42)
X_train=X_train.reshape([-1,120,120,1])
X_test=X_test.reshape([-1,120,120,1])


import tensorflow as tf
from tensorflow import keras 
from keras import layers
from keras import models

import random

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

Locmodel = models.Sequential()

Locmodel.add(layers.Conv2D(10, (3,3), activation="relu",
                           input_shape=(120,120,1)))

Locmodel.add(layers.MaxPooling2D((2,2)))


Locmodel.add(layers.Conv2D(64, (3,3), activation="relu"))

Locmodel.add(layers.MaxPooling2D((2,2)))

Locmodel.add(layers.Conv2D(64, (3,3), activation="relu"))

Locmodel.add(layers.Flatten())

Locmodel.add(layers.Dense(64, activation="relu"))

Locmodel.add(layers.Dense(32, activation="relu"))

Locmodel.add(layers.Dense(1, activation="sigmoid")) # sigmoid= range between 0 or 1   , 3 nodes =3 class then activation function should be softmax instead of sigmoid.

Locmodel.summary()


Locmodel.compile(optimizer="adam", loss="binary_crossentropy",metrics=["accuracy"])


# Locmodel.fit(X_train, y_train, epochs=2, batch_size=32)

# testLoss, testAccuracy = Locmodel.evaluate(X_test, y_test)

# print(testAccuracy)

# Locmodel.save_weights('./my_checkpoint')


# Define Flower client
import flwr as fl
class CifarClient(fl.client.NumPyClient):
  def get_parameters(self, config):
    return Locmodel.get_weights()

  def fit(self, parameters, config):
    Locmodel.set_weights(parameters)
    Locmodel.fit(X_train, y_train, epochs=1, batch_size=32)
    return Locmodel.get_weights(), len(X_train), {}

  def evaluate(self, parameters, config):
    Locmodel.set_weights(parameters)
    loss, accuracy = Locmodel.evaluate(X_test, y_test)
    return loss, len(X_test), {"accuracy": accuracy}

# Start Flower client
fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=CifarClient())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 118, 118, 10)      100       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 59, 59, 10)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 57, 57, 64)        5824      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 28, 28, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 26, 26, 64)        36928     
                                                                 
 flatten_1 (Flatten)         (None, 43264)            

INFO flwr 2023-08-13 19:02:51,958 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-08-13 19:02:51,979 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2023-08-13 19:02:51,982 | connection.py:39 | ChannelConnectivity.READY
DEBUG flwr 2023-08-13 19:05:00,033 | connection.py:113 | gRPC channel closed


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Stream removed"
	debug_error_string = "{"created":"@1691933700.032000000","description":"Error received from peer ipv4:127.0.0.1:8080","file":"src/core/lib/surface/call.cc","file_line":1064,"grpc_message":"Stream removed","grpc_status":2}"
>